In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])


train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

train_set = ImageFolder("/content/drive/MyDrive/gyartosor/train", transform = train_transform)
test_set = ImageFolder("/content/drive/MyDrive/gyartosor/test", transform = test_transform)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_set,
    batch_size=32,
    num_workers=4,
    shuffle=True,
)

test_loader = DataLoader(
    dataset=test_set,
    batch_size = 32,
    num_workers = 4,
    shuffle = True,
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        self.conv = nn.Conv2d(3, 18, kernel_size = 3, stride=1, padding=1) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.fc1 = nn.Linear(18 * 16 * 16, 64)
        self.fc2 = nn.Linear(64, 3) #itt az osztályok száma
        

    
    def forward(self, x):
        x = F.relu(self.conv(x))
        x = self.pool(x)
        x = x.view(-1, 18 * 16 * 16)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    
net = Net()
print(net)

Net(
  (conv): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)


In [ ]:
#importáljuk a már kész NN-t

from torchvision.models import squeezenet1_0

model = squeezenet1_0(pretrained=True)
print(model)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [ ]:
#módosítjuk az utolsó rétegét

n_classes = 10

model.num_classes = n_classes
model.classifier[1] = nn.Conv2d(512, n_classes, kernel_size=(1,1), stride=(1,1))

In [ ]:
def acc_count():
  correct, total = 0, 0
  predictions = []
  model.eval()
  for i, data in enumerate(test_loader, 0):
      inputs, labels = data
      outputs = model(inputs)
      _, predicted = torch.max(outputs.data, 1)
      predictions.append(outputs)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  print('Háló pontossága a test képeken %d %%' % (100 * correct / total))

In [ ]:
# from torch import nn, optim
import torch.nn as nn
import torch.optim as optim
#tréninmódba állítás
model.train()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

for epoch in range(20):
    for i, data in enumerate(train_loader, 0):
        # Get the inputs
        inputs, labels = data
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(epoch,'. epoch')
    acc_count()
print('Finished Training')

0 . epoch
Háló pontossága a test képeken 21 %
1 . epoch
Háló pontossága a test képeken 35 %
2 . epoch
Háló pontossága a test képeken 46 %
3 . epoch
Háló pontossága a test képeken 63 %
4 . epoch
Háló pontossága a test képeken 75 %
5 . epoch
Háló pontossága a test képeken 95 %
6 . epoch
Háló pontossága a test képeken 80 %
7 . epoch
Háló pontossága a test képeken 95 %
8 . epoch
Háló pontossága a test képeken 93 %
9 . epoch
Háló pontossága a test képeken 98 %
10 . epoch
Háló pontossága a test képeken 90 %
11 . epoch
Háló pontossága a test képeken 88 %
12 . epoch
Háló pontossága a test képeken 93 %
13 . epoch
Háló pontossága a test képeken 91 %
14 . epoch
Háló pontossága a test képeken 96 %
15 . epoch
Háló pontossága a test képeken 98 %
16 . epoch
Háló pontossága a test képeken 98 %
17 . epoch
Háló pontossága a test képeken 100 %
18 . epoch
Háló pontossága a test képeken 98 %
19 . epoch
Háló pontossága a test képeken 98 %
Finished Training


In [ ]:
acc_count()

Háló pontossága a test képeken 98 %
